.

Data_Engineering_TIL(20201204)

study program : T아카데미 - 아파치 스파크 입문과 활용

** URL : https://tacademy.skplanet.com/frontMain.action

### [학습내용]

step 1) 실습환경 구성


spark을 띄우기 위해서 spark cluster를 구성해야하는데 시간상 제플린 환경으로 실습한다. 제플린에서 spark context를 띄워서 실습해볼 수 있다.

참고로 spark을 설치한다는 것은 spark만 설치하면 끝나는게 아니라 yarn과 Hadoop을 설치를 해주고 연동해줘야 한다.


아래와 같은 spec으로 aws ec2 생성

In [ ]:
aws ec2 run-instances --image-id ami-03b42693dc6a7dc35 --count 1 --instance-type t3.medium --key-name pms-seoul-key --security-group-ids sg-xxxxxxxxxxxx --subnet-id subnet-xxxxxxxxxxxxx --associate-public-ip-address --tag-specifications 'ResourceType=instance,Tags=[{Key=Name,Value=pms-spark-test},{Key=owner,Value=pms},{Key=expiry-date,Value=2020-12-04}]' --block-device-mappings 'DeviceName=/dev/xvda,Ebs={VolumeSize=30,DeleteOnTermination=true}'

그런 다음에 해당 ec2로 ssh 접속해서 아래와 같은 명령어 실행

In [ ]:
[ec2-user@ip-10-0-1-170 ~]$ sudo yum update -y

[ec2-user@ip-10-0-1-170 ~]$ sudo amazon-linux-extras install docker -y
Installing docker
Loaded plugins: extras_suggestions, langpacks, priorities, update-motd
Cleaning repos: amzn2-core amzn2extra-docker
12 metadata files removed
4 sqlite files removed
0 metadata files removed
Loaded plugins: extras_suggestions, langpacks, priorities, update-motd
amzn2-core                                                                                    | 3.7 kB  00:00:00
amzn2extra-docker                                                                             | 3.0 kB  00:00:00
(1/5): amzn2-core/2/x86_64/group_gz                                                           | 2.5 kB  00:00:00
(2/5): amzn2-core/2/x86_64/updateinfo                                                         | 309 kB  00:00:00
(3/5): amzn2extra-docker/2/x86_64/updateinfo                                                  |   76 B  00:00:00
(4/5): amzn2extra-docker/2/x86_64/primary_db                                                  |  74 kB  00:00:00
(5/5): amzn2-core/2/x86_64/primary_db                                                         |  47 MB  00:00:00
Resolving Dependencies
--> Running transaction check
---> Package docker.x86_64 0:19.03.13ce-1.amzn2 will be installed
--> Processing Dependency: runc >= 1.0.0 for package: docker-19.03.13ce-1.amzn2.x86_64
--> Processing Dependency: containerd >= 1.3.2 for package: docker-19.03.13ce-1.amzn2.x86_64
--> Processing Dependency: pigz for package: docker-19.03.13ce-1.amzn2.x86_64
--> Processing Dependency: libcgroup for package: docker-19.03.13ce-1.amzn2.x86_64
--> Running transaction check
---> Package containerd.x86_64 0:1.4.1-2.amzn2 will be installed
---> Package libcgroup.x86_64 0:0.41-21.amzn2 will be installed
---> Package pigz.x86_64 0:2.3.4-1.amzn2.0.1 will be installed
---> Package runc.x86_64 0:1.0.0-0.1.20200826.gitff819c7.amzn2 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

=====================================================================================================================
 Package             Arch            Version                                        Repository                  Size
=====================================================================================================================
Installing:
 docker              x86_64          19.03.13ce-1.amzn2                             amzn2extra-docker           37 M
Installing for dependencies:
 containerd          x86_64          1.4.1-2.amzn2                                  amzn2extra-docker           24 M
 libcgroup           x86_64          0.41-21.amzn2                                  amzn2-core                  66 k
 pigz                x86_64          2.3.4-1.amzn2.0.1                              amzn2-core                  81 k
 runc                x86_64          1.0.0-0.1.20200826.gitff819c7.amzn2            amzn2extra-docker          3.7 M

Transaction Summary
=====================================================================================================================
Install  1 Package (+4 Dependent packages)

Total download size: 65 M
Installed size: 270 M
Downloading packages:
(1/5): libcgroup-0.41-21.amzn2.x86_64.rpm                                                     |  66 kB  00:00:00
(2/5): pigz-2.3.4-1.amzn2.0.1.x86_64.rpm                                                      |  81 kB  00:00:00
(3/5): containerd-1.4.1-2.amzn2.x86_64.rpm                                                    |  24 MB  00:00:00
(4/5): runc-1.0.0-0.1.20200826.gitff819c7.amzn2.x86_64.rpm                                    | 3.7 MB  00:00:00
(5/5): docker-19.03.13ce-1.amzn2.x86_64.rpm                                                   |  37 MB  00:00:00
---------------------------------------------------------------------------------------------------------------------
Total                                                                                103 MB/s |  65 MB  00:00:00
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : runc-1.0.0-0.1.20200826.gitff819c7.amzn2.x86_64                                                   1/5
  Installing : containerd-1.4.1-2.amzn2.x86_64                                                                   2/5
  Installing : pigz-2.3.4-1.amzn2.0.1.x86_64                                                                     3/5
  Installing : libcgroup-0.41-21.amzn2.x86_64                                                                    4/5
  Installing : docker-19.03.13ce-1.amzn2.x86_64                                                                  5/5
  Verifying  : docker-19.03.13ce-1.amzn2.x86_64                                                                  1/5
  Verifying  : libcgroup-0.41-21.amzn2.x86_64                                                                    2/5
  Verifying  : pigz-2.3.4-1.amzn2.0.1.x86_64                                                                     3/5
  Verifying  : containerd-1.4.1-2.amzn2.x86_64                                                                   4/5
  Verifying  : runc-1.0.0-0.1.20200826.gitff819c7.amzn2.x86_64                                                   5/5

Installed:
  docker.x86_64 0:19.03.13ce-1.amzn2

Dependency Installed:
  containerd.x86_64 0:1.4.1-2.amzn2                 libcgroup.x86_64 0:0.41-21.amzn2 pigz.x86_64 0:2.3.4-1.amzn2.0.1
  runc.x86_64 0:1.0.0-0.1.20200826.gitff819c7.amzn2

Complete!
  0  ansible2                 available    \
        [ =2.4.2  =2.4.6  =2.8  =stable ]
  2  httpd_modules            available    [ =1.0  =stable ]
  3  memcached1.5             available    \
        [ =1.5.1  =1.5.16  =1.5.17 ]
  5  postgresql9.6            available    \
        [ =9.6.6  =9.6.8  =stable ]
  6  postgresql10             available    [ =10  =stable ]
  8  redis4.0                 available    \
        [ =4.0.5  =4.0.10  =stable ]
  9  R3.4                     available    [ =3.4.3  =stable ]
 10  rust1                    available    \
        [ =1.22.1  =1.26.0  =1.26.1  =1.27.2  =1.31.0  =1.38.0
          =stable ]
 11  vim                      available    [ =8.0  =stable ]
 15  php7.2                   available    \
        [ =7.2.0  =7.2.4  =7.2.5  =7.2.8  =7.2.11  =7.2.13  =7.2.14
          =7.2.16  =7.2.17  =7.2.19  =7.2.21  =7.2.22  =7.2.23
          =7.2.24  =7.2.26  =stable ]
 17  lamp-mariadb10.2-php7.2  available    \
        [ =10.2.10_7.2.0  =10.2.10_7.2.4  =10.2.10_7.2.5
          =10.2.10_7.2.8  =10.2.10_7.2.11  =10.2.10_7.2.13
          =10.2.10_7.2.14  =10.2.10_7.2.16  =10.2.10_7.2.17
          =10.2.10_7.2.19  =10.2.10_7.2.22  =10.2.10_7.2.23
          =10.2.10_7.2.24  =stable ]
 18  libreoffice              available    \
        [ =5.0.6.2_15  =5.3.6.1  =stable ]
 19  gimp                     available    [ =2.8.22 ]
 20  docker=latest            enabled      \
        [ =17.12.1  =18.03.1  =18.06.1  =18.09.9  =stable ]
 21  mate-desktop1.x          available    \
        [ =1.19.0  =1.20.0  =stable ]
 22  GraphicsMagick1.3        available    \
        [ =1.3.29  =1.3.32  =1.3.34  =stable ]
 23  tomcat8.5                available    \
        [ =8.5.31  =8.5.32  =8.5.38  =8.5.40  =8.5.42  =8.5.50
          =stable ]
 24  epel                     available    [ =7.11  =stable ]
 25  testing                  available    [ =1.0  =stable ]
 26  ecs                      available    [ =stable ]
 27  corretto8                available    \
        [ =1.8.0_192  =1.8.0_202  =1.8.0_212  =1.8.0_222  =1.8.0_232
          =1.8.0_242  =stable ]
 28  firecracker              available    [ =0.11  =stable ]
 29  golang1.11               available    \
        [ =1.11.3  =1.11.11  =1.11.13  =stable ]
 30  squid4                   available    [ =4  =stable ]
 31  php7.3                   available    \
        [ =7.3.2  =7.3.3  =7.3.4  =7.3.6  =7.3.8  =7.3.9  =7.3.10
          =7.3.11  =7.3.13  =stable ]
 32  lustre2.10               available    \
        [ =2.10.5  =2.10.8  =stable ]
 33  java-openjdk11           available    [ =11  =stable ]
 34  lynis                    available    [ =stable ]
 35  kernel-ng                available    [ =stable ]
 36  BCC                      available    [ =0.x  =stable ]
 37  mono                     available    [ =5.x  =stable ]
 38  nginx1                   available    [ =stable ]
 39  ruby2.6                  available    [ =2.6  =stable ]
 40  mock                     available    [ =stable ]
 41  postgresql11             available    [ =11  =stable ]
 42  php7.4                   available    [ =stable ]
 43  livepatch                available    [ =stable ]
 44  python3.8                available    [ =stable ]
 45  haproxy2                 available    [ =stable ]
 46  collectd                 available    [ =stable ]
 47  aws-nitro-enclaves-cli   available    [ =stable ]
 48  R4                       available    [ =stable ]

[ec2-user@ip-10-0-1-170 ~]$ sudo service docker start
Redirecting to /bin/systemctl start docker.service

[ec2-user@ip-10-0-1-170 ~]$ sudo usermod -a -G docker ec2-user

[ec2-user@ip-10-0-1-170 ~]$ exit

터미널 로그아웃 후 다시 ec2로 재접속하고 아래와 같이 명령어를 실행했을때 sudo 명령어를 안줘도 되는지 확인해본다.

In [ ]:
[ec2-user@ip-10-0-1-170 ~]$ docker info
Client:
 Debug Mode: false

Server:
 Containers: 0
  Running: 0
  Paused: 0
  Stopped: 0
 Images: 0
 Server Version: 19.03.13-ce
 Storage Driver: overlay2
  Backing Filesystem: xfs
  Supports d_type: true
  Native Overlay Diff: true
 Logging Driver: json-file
 Cgroup Driver: cgroupfs
 Plugins:
  Volume: local
  Network: bridge host ipvlan macvlan null overlay
  Log: awslogs fluentd gcplogs gelf journald json-file local logentries splunk syslog
 Swarm: inactive
 Runtimes: runc
 Default Runtime: runc
 Init Binary: docker-init
 containerd version: c623d1b36f09f8ef6536a057bd658b3aa8632828
 runc version: ff819c7e9184c13b7c2607fe6c30ae19403a7aff
 init version: de40ad0 (expected: fec3683)
 Security Options:
  seccomp
   Profile: default
 Kernel Version: 4.14.193-149.317.amzn2.x86_64
 Operating System: Amazon Linux 2
 OSType: linux
 Architecture: x86_64
 CPUs: 2
 Total Memory: 3.794GiB
 Name: ip-10-0-1-170.ap-northeast-2.compute.internal
 ID: SPCF:Y75R:B22D:BLRW:ELZ5:3O4B:HPDJ:R3JI:DSQM:W74R:TRA2:BXWE
 Docker Root Dir: /var/lib/docker
 Debug Mode: false
 Registry: https://index.docker.io/v1/
 Labels:
 Experimental: false
 Insecure Registries:
  127.0.0.0/8
 Live Restore Enabled: false

아래와 같은 docker run 명령어를 실행해서 제플린 개발환경을 구성한다.

`docker run -p 4040:4040 -p 8080:8080 --privileged=true -v $PWD/logs:/logs -v $PWD/notebook:/notebook -e ZEPPELIN_NOTEBOOK_DIR='/notebook' -e ZEPPELIN_LOG_DIR='/logs' -d apache/zeppelin:0.8.1 /zeppelin/bin/zeppelin.sh`

In [ ]:
[ec2-user@ip-10-0-1-170 ~]$ docker run -p 4040:4040 -p 8080:8080 --privileged=true -v $PWD/logs:/logs -v $PWD/notebook:/notebook \
> -e ZEPPELIN_NOTEBOOK_DIR='/notebook' \
> -e ZEPPELIN_LOG_DIR='/logs' \
> -d apache/zeppelin:0.8.1 \
> /zeppelin/bin/zeppelin.sh
Unable to find image 'apache/zeppelin:0.8.1' locally
0.8.1: Pulling from apache/zeppelin
7b722c1070cd: Pull complete
5fbf74db61f1: Pull complete
ed41cb72e5c9: Pull complete
7ea47a67709e: Pull complete
7ba34fd9f5e0: Pull complete
8f2f09b83582: Pull complete
40260f0a8f69: Pull complete
48946af5572c: Pull complete
8b38acee7e8d: Pull complete
a806f41d7e41: Pull complete
7dcaf396dead: Pull complete
8db355f40e66: Pull complete
Digest: sha256:a3a90ec1579f5171ebac565e739547b885ed75efc1ec7581128ec1033a4496cb
Status: Downloaded newer image for apache/zeppelin:0.8.1
9276b4344e403414c354ec41816abcd6005e637836d31f691319323b672fe279

[ec2-user@ip-10-0-1-170 ~]$ docker ps -a
CONTAINER ID        IMAGE                   COMMAND                  CREATED             STATUS              PORTS                                            NAMES
9276b4344e40        apache/zeppelin:0.8.1   "/usr/bin/tini -- /z…"   7 seconds ago       Up 5 seconds        0.0.0.0:4040->4040/tcp, 0.0.0.0:8080->8080/tcp   confident_swirles

위와 같이 정상적으로 도커를 실행하고 나면 웹브라우저를 열고 `[ec2 public ip]:8080` 으로 접속해서 제플린으로 정상 접근가능한지 확인해본다.

접속한 다음에 `Create new note`을 클릭해서 spark 노트북을 새로 만들어준다. 그런 다음에 아래 그림과 같이 첫번째 블럭에 spark 문자열을 입력후 실행해본다. 실행하면 sparksession이 구동되는 것을 확인할 수 있다.

![1](https://user-images.githubusercontent.com/41605276/101123798-f3b92e00-3638-11eb-8be0-71e23ed84eae.PNG)

그런 다음에 웹브라우저를 하나 새로열고 `[ec2 public ip]:4040`를 입력해서 아래 그림과 같이 spark ui에 정상접속하는지 확인해본다.


![2](https://user-images.githubusercontent.com/41605276/101123952-4e528a00-3639-11eb-9a9e-889d83083460.PNG)

생성한 제플린 노트북에서 아래와 같은 코드로 실습을 해본다.

In [ ]:
%spark
//sparksession driver process,어플리케이션 기준 
spark


%spark
//한개의 컬럼과 1000개의 row(0~999)를 생성한 데이터 프레임을 만든다
val myRange = spark.range(1000).toDF("number")


%spark
//transformation - narrow dependency
val divisBy2 = myRange.where("number % 2 = 0")

%spark
//action
divisBy2.count()


%sh
wget https://raw.githubusercontent.com/databricks/Spark-The-Definitive-Guide/master/data/flight-data/csv/2015-summary.csv
//데이터 다운로드

%sh
head /zeppelin/2015-summary.csv
//데이터 확인


%spark
val flightData2015 = spark.read.option("inferSchema", "true").option("header", "true").csv("/zeppelin/2015-summary.csv")
//CSV데이터 데이터 프레임으로 바로 읽기, visit to http://localhost:4040


%spark
flightData2015.take(3)
//3건만 가져오기

%spark
flightData2015.sort("count").explain()
//실행계획 확인하기

%spark
flightData2015.sort("count").take(2)
//count필드로 정렬해서 2개 가져오기


%spark
flightData2015.createOrReplaceTempView("flight_data_2015")
//SQL을 사용하기 위해 임시 view만들기

%spark
val sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")
//SQL로DEST_COUNTRY_NAME 기준으로 몇건인지 확인하는 spark sql실행하기

%spark
sqlWay.describe().show()
//데이터 프레임 통계 보기


%spark
val dataFrameWay = flightData2015
  .groupBy('DEST_COUNTRY_NAME)
  .count.explain
//데이터 프레임으로 실행계획 확인하기

%spark
sqlWay.explain
//sql과 dataframe실행계획 비교하기

%spark
spark.sql("SELECT max(count) from flight_data_2015").take(1)
//sql질의로 max값 가져와서 take로 한건만 확인하기

%spark
import org.apache.spark.sql.functions.max

flightData2015.select(max("count")).take(1)
//데이터 프레임에서 max함수 이용하여 count의 최대값 가져오기

%spark
val maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

//spark sql로 DEST_COUNTRY_NAME 을 집계연산을 수행하여 합계를 구하여 큰 순서대로 5건 확인하기

%spark
maxSql.explain
maxSql.show()
//sql결과 확인하기


%spark
//동일한 쿼리를 dataframe으로 구현해 봅니다.  End-to-End example
flightData2015
.groupBy("DEST_COUNTRY_NAME")
.sum("count")
.withColumnRenamed("sum(count)", "destination_total")
.sort(desc("destination_total"))
.limit(5)
.explain


%spark
flightData2015
.groupBy("DEST_COUNTRY_NAME")
.sum("count")
.withColumnRenamed("sum(count)", "destination_total")
.sort(desc("destination_total"))
.limit(5)
.show()


%spark
//임의의 값을 가지고 데이터 프레임 생성하기
val data = Seq(("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"),
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"),
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"),
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico"))

import spark.sqlContext.implicits._
val df = data.toDF("Product","Amount","Country")
df.show()

//각 제품의 각 국가로 수출 된 총 금액을 얻으려면 제품 별 그룹화, 국가 별 피봇 팅 및 금액 합계를 얻어온다
val pivotDF = df.groupBy("Product").pivot("Country").sum("Amount")
pivotDF.show()

//spark2.0에서 성능향상을 위해 pivot대상열을 선언함 
val countries = Seq("USA","China","Canada","Mexico")
val pivotDF = df.groupBy("Product").pivot("Country", countries).sum("Amount")
pivotDF.show()

//성능 향상을 위해 두단계 집계를 사용함 
val pivotDF = df.groupBy("Product","Country")
      .sum("Amount")
      .groupBy("Product")
      .pivot("Country")
      .sum("sum(Amount)")
pivotDF.show()


//stack기능을 이용하여 주요 국가에 대해 unpivot을 수행
val unPivotDF = pivotDF.select($"Product",
expr("stack(4, 'Canada', Canada, 'China', China, 'Mexico', Mexico, 'USA', USA) as (Country,Total)"))
.where("Total is not null")
unPivotDF.show()

https://www.youtube.com/watch?v=XwcDKo0Hce0&feature=youtu.be

3시간 21분 8초